In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


Starting from here

In [ ]:
import pandas as pd
df_participants = pd.read_pickle('/content/drive/MyDrive/TD-BRAIN/TDBRAIN_participants_V2_data/df_participants.pkl')
print(f'all participants: {df_participants.shape}')

all participants: (370, 13)


In [ ]:
df_participants.sample(5)

,participants_ID,DISC/REP,indication,formal_status,Dataset,age,gender,sessID,nrSessions,EC,EO,diagnosis,diagnosis_group
257,sub-87976641,DISCOVERY,HEALTHY,HEALTHY,NaN,NaN,0,1,1,True,True,HEALTHY,HEALTHY
940,sub-88046437,DISCOVERY,MDD,MDD,MDD-rTMS,39.54,1,1,1,True,True,MDD,MDD
867,sub-88041665,DISCOVERY,MDD,MDD,MDD-rTMS,43.89,0,1,1,True,True,MDD,MDD
621,sub-88021321,DISCOVERY,MDD,MDD,MDD-rTMS,28.28,1,1,1,True,True,MDD,MDD
426,sub-88010981,DISCOVERY,MDD,MDD,MDD-rTMS,47.09,0,1,2,True,True,MDD,MDD


In [ ]:
labels_df = df_participants[["participants_ID", "diagnosis"]]
labels_df

,participants_ID,diagnosis
245,sub-87974617,HEALTHY
246,sub-87974621,HEALTHY
247,sub-87974665,HEALTHY
248,sub-87974709,HEALTHY
249,sub-87974841,HEALTHY
...,...,...
1320,sub-88076941,MDD
1326,sub-88077345,MDD
1328,sub-88077525,MDD
1329,sub-88077569,MDD


In [ ]:
pip install mne

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 46.3 MB/s eta 0:00:00


In [ ]:
import scipy
import os
import tensorflow as tf
import mne
import numpy as np

sample_ids = np.array(labels_df['participants_ID'])

# Define only the EEG channels
ch_types_eeg = ['eeg'] * 26
ch_names_eeg = ['Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FC3', 'FCz', 'FC4', 'T7', 'C3', 'Cz',
                'C4', 'T8', 'CP3', 'CPz', 'CP4', 'P7', 'P3', 'Pz', 'P4', 'P8', 'O1', 'Oz', 'O2']

epoch_data_list = []
counter = 0
labels_samples_id = []

for sample_id in labels_df["participants_ID"]:
    npy_dir = f"/content/drive/MyDrive/TD-BRAIN/preprocessed/{sample_id}/ses-1/eeg/"
    if os.path.exists(npy_dir):
        for file in os.listdir(npy_dir):
            file_path = os.path.join(npy_dir, file)
            if os.path.isfile(file_path) and "EC" in file and '.npy' in file and 'BAD' not in file:
                preprocessed_eeg = np.load(file_path, allow_pickle=True)

                data_shape = preprocessed_eeg['data'].shape
                print(f"Sample ID: {sample_id}, File: {file}, Data Shape: {data_shape}")

                # Squeeze the data to remove the first dimension if it is 1
                data_eeg = np.squeeze(preprocessed_eeg['data'])

                # Check if data has at least 26 channels after squeezing
                if data_eeg.shape[0] >= 26:
                    data_eeg = data_eeg[:26, :]  # Select only the first 26 channels

                    info = mne.create_info(ch_names=ch_names_eeg, ch_types=ch_types_eeg, sfreq=500)
                    raw = mne.io.RawArray(data_eeg, info)
                    epoch_data = mne.make_fixed_length_epochs(raw, duration=9.95, overlap=0)

                    # labels
                    index = labels_df.loc[labels_df['participants_ID'] == sample_id].index
                    diagnosis = labels_df.loc[index, "diagnosis"]
                    diagnosis_value = [value for value in diagnosis][0]

                    for i in range(12):
                        epoch_data_list.append(epoch_data[i].get_data())
                        labels_samples_id.append([sample_id, diagnosis_value])
                        counter += 1
                    break
                else:
                    print(f"Skipping sample-id: {sample_id}. The EEG data does not have at least 26 channels.")

epoch_data_array = np.stack(epoch_data_list)



Streaminguitvoer ingekort tot de laatste 5000 regels.
Using data from preloaded Raw for 1 events and 4975 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 1 events and 4975 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 1 events and 4975 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 1 events and 4975 original time points ...
0 bad epochs dropped
Sample ID: sub-88046033, File: sub-88046033_ses-1_task-restEC_eeg_csv_120.016s.npy, Data Shape: (1, 32, 60008)
Creating RawArray with float64 data, n_channels=26, n_times=60008
    Range : 0 ... 60007 =      0.000 ...   120.014 secs
Ready.
Not setting metadata
12 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1 events and 4975 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 1 events and 4975 original time points ...
0 bad epochs dropped
Using data f

In [ ]:
counter

4356

In [ ]:
print(len(labels_samples_id))
print(labels_samples_id)

4356
[['sub-87974617', 'HEALTHY'], ['sub-87974617', 'HEALTHY'], ['sub-87974617', 'HEALTHY'], ['sub-87974617', 'HEALTHY'], ['sub-87974617', 'HEALTHY'], ['sub-87974617', 'HEALTHY'], ['sub-87974617', 'HEALTHY'], ['sub-87974617', 'HEALTHY'], ['sub-87974617', 'HEALTHY'], ['sub-87974617', 'HEALTHY'], ['sub-87974617', 'HEALTHY'], ['sub-87974617', 'HEALTHY'], ['sub-87974621', 'HEALTHY'], ['sub-87974621', 'HEALTHY'], ['sub-87974621', 'HEALTHY'], ['sub-87974621', 'HEALTHY'], ['sub-87974621', 'HEALTHY'], ['sub-87974621', 'HEALTHY'], ['sub-87974621', 'HEALTHY'], ['sub-87974621', 'HEALTHY'], ['sub-87974621', 'HEALTHY'], ['sub-87974621', 'HEALTHY'], ['sub-87974621', 'HEALTHY'], ['sub-87974621', 'HEALTHY'], ['sub-87974665', 'HEALTHY'], ['sub-87974665', 'HEALTHY'], ['sub-87974665', 'HEALTHY'], ['sub-87974665', 'HEALTHY'], ['sub-87974665', 'HEALTHY'], ['sub-87974665', 'HEALTHY'], ['sub-87974665', 'HEALTHY'], ['sub-87974665', 'HEALTHY'], ['sub-87974665', 'HEALTHY'], ['sub-87974665', 'HEALTHY'], ['sub-87

In [ ]:
print(epoch_data_array.shape)
labels_samples_array = np.array(labels_samples_id)
labels_samples_array.shape

(4356, 1, 26, 4975)


(4356, 2)

In [ ]:
# Initialize an empty list to store data
data = []

# Loop over the array
for i in range(0, 4356, 12):
    # Append a tuple with participant ID and diagnosis to the list
    data.append((labels_samples_array[i][0], labels_samples_array[i][1]))

# Create the DataFrame from the list of tuples
labels_df = pd.DataFrame(data, columns=['participants_ID', 'diagnosis'])

# Display the DataFrame
# print(labels_df)

In [ ]:
file_path = "/content/drive/MyDrive/TD-BRAIN/training_data/data/unique_labels.csv"

# Save the DataFrame to CSV
labels_df.to_csv(file_path, index=False)

In [ ]:
save_dir = "/content/drive/MyDrive/TD-BRAIN/training_data/data/EC_26"
os.makedirs(save_dir, exist_ok=True)
np.save(os.path.join(save_dir, "labels_data.npy"), labels_samples_array)

In [ ]:
first_epoch_data = epoch_data[0].get_data()
print(epoch_data_array.shape)

Using data from preloaded Raw for 1 events and 4975 original time points ...
0 bad epochs dropped
(4356, 1, 26, 4975)


In [ ]:
# Save EEG data and channels as NumPy arrays
save_dir = "/content/drive/MyDrive/TD-BRAIN/training_data/data/EC_26"
os.makedirs(save_dir, exist_ok=True)
np.save(os.path.join(save_dir, "raw_epoch_eeg_data.npy"), epoch_data_array)

Normalizing the Data

In [ ]:
from sklearn.preprocessing import StandardScaler

def scale_epoch_data(epoch_data_array):

    reshaped_data = epoch_data_array.reshape(-1, epoch_data_array.shape[-1])

    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(reshaped_data)


    scaled_epoch_data_array = scaled_data.reshape(epoch_data_array.shape)

    return scaled_epoch_data_array

In [ ]:
# Scale the epoch data array
scaled_epoch_data_array = scale_epoch_data(epoch_data_array)

In [ ]:
print(scaled_epoch_data_array[0])

In [ ]:
# print(epoch_data_array[0])

In [ ]:
# Save EEG data and channels as NumPy arrays
save_dir = "/content/drive/MyDrive/TD-BRAIN/training_data/data/EC_26"
os.makedirs(save_dir, exist_ok=True)
np.save(os.path.join(save_dir, "normalized_epoch_eeg_data.npy"), epoch_data_array)